In [1]:
import spotipy
#import json
from spotipy.oauth2 import SpotifyClientCredentials
from config import *
import pandas as pd
import numpy as np
import re
from time import sleep
from random import randint

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

In [3]:
top100 = pd.read_csv('top100.csv')
not_hot_songs = pd.read_csv('not_hot_songs.csv')

In [4]:
# Create a function to search a given song in the Spotify API: search_song(). 
# Take into account that sometimes Spotify's API will return several matches for the same song title 
# (different artists, a different album of the same artist, version of the song,...etc). 
# Then it will be nice to display a list of outputs to the user and let him/her select which 
# is the right match. Once the desired song is located, the function should return the href/id/uri 
# of the song to the code (not to the user).

In [31]:
def search_song(df):
    
        print('Querying IDs...')
        
        steps=[i for i in range(50,df.shape[0],50)]
    
        singer_0 = df['singer'].str.split('&|X|/|Featuring',
                                                expand=True).add_prefix('singer_')['singer_0'].tolist()
        song = df['song'].to_list()
        ids = []
        
        for n,i in enumerate(song):
            
            results = sp.search(q='track:{} artist:{}'.format(song[n],singer_0[n]), limit=1)

            if results['tracks']['total']>0:
                x = results['tracks']['items'][0]['id'] 
                ids.append(x)
            else: 
                x = ''
                ids.append(x)
                
            if n in steps:
                seconds_to_wait = randint(1,10)
                print("Waiting {} seconds before querying next 50 songs! (n={})".format(seconds_to_wait,n))
                sleep(seconds_to_wait)
                
            
        df['id'] = ids
        
        print('Querying IDs... Done\n')        
        return df
    

In [6]:
# Create a function "get_audio_features(list_of_songs)" to obtain the audio features of a given list 
# of songs (the content of list_of_songs can be the href/id/uri). Then, use this function to create 
# a Pandas Dataframe with the audio features of the list of songs. Hint: create a dictionary with 
# the song's audio features as keys and an empty list as values. Then fill in the lists with 
# the corresponding audio features of each song. Finally, create your data frame from the dictionary. 
# Bear in mind the following: This API has a restriction on the number of requests per minute, therefore, 
# consider launching the search using smaller groups of songs.

In [125]:
def get_audio_features(list_of_songs):
    
    print('Querying audio features...')
    
    dic = {}
    for key in sp.audio_features('4yzUTT6pCLKlaWlNwe4XM8')[0].keys():
        dic[key]=[]
       
    steps=[j for j in range(50,len(list_of_songs),50)]
    
    for n,i in enumerate(list_of_songs):
        if i != '': ## some songs have not ids
                a = sp.audio_features(i)
                if a != [None]: ## some songs have id but not audio features
                    for key in a[0].keys():
                        dic[key].append(a[0][key])
                else:
                    for key in dic.keys():
                        dic[key].append('')
        else:
            for key in dic.keys():
                dic[key].append('')
        if n in steps:
            seconds_to_wait = randint(1,10)
            print("Waiting {} seconds before querying next 50 songs! (n={})".format(seconds_to_wait,n))
            sleep(seconds_to_wait)
            
    
    df = pd.DataFrame(dic)    
    
    print('Querying audio features... Done')
    
    return df


In [8]:
#list_of_songs = top100['id'].tolist()
#top100af = get_audio_features(list_of_songs)
#top100af

In [9]:
# Once the previous function has been created, create another function 
# "add_audio_features(df, audio_features_df)" to concat a given data frame with the data frame 
# containing the audio features alongside any other desired info, and return the extended data frame.

In [10]:
def add_audio_features(df, audio_features_df):
    x = pd.concat([df,audio_features_df], axis =1)
    # x.to_csv('top100.csv')
    return x

In [29]:
def all_in(df):
    search_song(df)
    list_of_songs = df['id'].tolist()
    df_af         = get_audio_features(list_of_songs)
    df_concat     = add_audio_features(df,df_af)
    return df_concat

data = all_in(top100)
data.to_csv('top100_a.csv')


Querying IDs...
Waiting 11 seconds before querying next 50 songs! (n=50)
Querying IDs... Done

Querying audio features...
Waiting 28 seconds before querying next 50 songs! (n=50)
Querying audio features... Done


In [131]:
def all_in(df):
    #search_song(df)
    list_of_songs = df['id'].tolist()
    df_af         = get_audio_features(list_of_songs)
    df_concat     = add_audio_features(df,df_af)
    return df_concat

data = all_in(not_hot_songs)
data.to_csv('not_hot_songs_a.csv')

Querying audio features...
Waiting 8 seconds before querying next 50 songs! (n=50)
Waiting 7 seconds before querying next 50 songs! (n=100)
Waiting 8 seconds before querying next 50 songs! (n=150)
Waiting 2 seconds before querying next 50 songs! (n=200)
Waiting 1 seconds before querying next 50 songs! (n=250)
Waiting 1 seconds before querying next 50 songs! (n=300)
Waiting 5 seconds before querying next 50 songs! (n=350)
Waiting 9 seconds before querying next 50 songs! (n=400)
Waiting 6 seconds before querying next 50 songs! (n=450)
Waiting 3 seconds before querying next 50 songs! (n=500)
Waiting 6 seconds before querying next 50 songs! (n=550)
Waiting 4 seconds before querying next 50 songs! (n=600)
Waiting 5 seconds before querying next 50 songs! (n=650)
Waiting 7 seconds before querying next 50 songs! (n=700)
Waiting 6 seconds before querying next 50 songs! (n=750)
Waiting 3 seconds before querying next 50 songs! (n=800)
Waiting 7 seconds before querying next 50 songs! (n=850)
Waiti